In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from math import ceil
import os

In [2]:
NTURN = [128,256,512,1024]
nTurn = NTURN[0]
pipe_radius = 0.02*np.sqrt(2.0)
npt = 3834

# 1. Preparation

In [3]:
beam, lattice = impact.readInputFile('test.in.1024.forward')
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

reading ImpactZ input file (test.in.1024.forward)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


### TBT

In [4]:
TBT = impact.getElem('TBT')
TBT.pID_begin = 1
TBT.pID_end   = npt
TBT.file_id = nTurn+100000
lattice.insert(1,TBT)

TBT_Inv = impact.getElem('TBT_integral')
TBT_Inv.file_id = dTune
TBT_Inv.betx = betx
TBT_Inv.alfx = alfx
TBT_Inv.strength_t = 0.4
TBT_Inv.transverse_scale_c = 0.01
lattice.insert(2,TBT_Inv)

### backward lattice

In [5]:
latticeB = copy(lattice[::-1])
latticeB.insert(0,copy(lattice[0]))
latticeB.pop(1)
latticeB[0].turns = nTurn

for i in range(len(latticeB)):
    if 'length' in latticeB[i]:
        latticeB[i].length = -latticeB[i].length
    if latticeB[i].type == 'dipole':
        latticeB[i].bending_angle            =-latticeB[i].bending_angle
        latticeB[i].fringe_field_integration =-latticeB[i].fringe_field_integration

In [6]:
i = 34
print(latticeB[-i])
print(lattice[i])


                   length: -0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: -1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.02 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: -0.5 [1]
                     type: 'dipole'

                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.02 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'


### write Impact input file

In [7]:
beam.distribution.distribution_type = 'ReadFile_binary'
beam.distribution.file_id = nTurn
# beam.nCore_y = nCore_y
# beam.nCore_z = nCore_z
impact.writeInputFile(beam,latticeB,'test.in.'+str(nTurn)+'.backward')
# impact.writeInputFile(beam,latticeB)
# impact.run(beam)